Assuming you have installed the AtLAST sensitivity calculator module, import it along with useful astropy functions

In [21]:
from atlast_sc.calculator import Calculator
import astropy.units as u  # for ensuring units are treated properly
import astropy.constants as const # to make use of the astropy constants package
import numpy as np

Set up the band structure - here assuming something similar to the ALMA bands

In [31]:
band_inds=np.arange(3,11)
nband=len(band_inds)
band_freq=np.zeros((nband,2))
band_freq[0,0]=84
band_freq[0,1]=116
band_freq[1,0]=125
band_freq[1,1]=163
band_freq[2,0]=163
band_freq[2,1]=211
band_freq[3,0]=211
band_freq[3,1]=275
band_freq[4,0]=275
band_freq[4,1]=373
band_freq[5,0]=385
band_freq[5,1]=500
band_freq[6,0]=602
band_freq[6,1]=720
band_freq[7,0]=787
band_freq[7,1]=950

Create an instance of the sensitivity calculator and adjust observing parameters.  Use 1 hr as a reference integration time.

In [32]:
calculator = Calculator()
integration_time = 3600 *u.s
calculator.t_int = integration_time

Get the telescope radius from the calculator

In [33]:
radius = calculator.instrument_setup.dish_radius.value
print(radius)

25.0 m


Write out some estimated sensitivities and beam sizes and/or store them for later

In [34]:
fout=open('sensitivity_calculations.txt', 'w')
fout.write('%-6s %10s %10s %10s %10s %10s\n' % ('# Band', 'nu1', 'nu2', 'beam', 'int_time', 'sensitivity'))
fout.write('# Beam size in arcsec; integration time in seconds; sensitivity in mJy/beam\n')
sens_per_hour=np.zeros((nband))
beam_size=np.zeros((nband))
for i, b in enumerate(band_freq):
    calculator.obs_freq = np.sum(b)/2*u.GHz
    calculator.bandwidth = (b[1]-b[0])*u.GHz

    # pull the observing frequency from the user inputs
    freq = calculator.user_input.obs_freq.value

    theta = ((1.2* const.c / (freq * (2*radius) ))*u.radian).to('arcsec')
    print(str(i)+f' {freq:0.2f} {theta:0.2f}')
    beam_size[i]=theta.value

    # point source sensitivity
    calculated_sensitivity = calculator.calculate_sensitivity(integration_time).to(u.mJy)
    print("Sensitivity: {:0.5f} per beam for an integration time of {:0.2f} ".format(calculated_sensitivity, integration_time))
    fout.write('%-6d %10.2f %10.2f %10.2f %10d %10.6f\n' % (i+3, b[0], b[1], theta.value, integration_time.value, calculated_sensitivity.value))
    sens_per_hour[i]=calculated_sensitivity.value
    
fout.close()


0 100.00 GHz 14.84 arcsec
Sensitivity: 0.00580 mJy per beam for an integration time of 3600.00 s 
1 144.00 GHz 10.31 arcsec
Sensitivity: 0.00676 mJy per beam for an integration time of 3600.00 s 
2 187.00 GHz 7.94 arcsec
Sensitivity: 0.00944 mJy per beam for an integration time of 3600.00 s 
3 243.00 GHz 6.11 arcsec
Sensitivity: 0.00820 mJy per beam for an integration time of 3600.00 s 
4 324.00 GHz 4.58 arcsec
Sensitivity: 0.03205 mJy per beam for an integration time of 3600.00 s 
5 442.50 GHz 3.35 arcsec
Sensitivity: 0.05861 mJy per beam for an integration time of 3600.00 s 
6 661.00 GHz 2.25 arcsec
Sensitivity: 0.04743 mJy per beam for an integration time of 3600.00 s 
7 868.50 GHz 1.71 arcsec
Sensitivity: 0.07525 mJy per beam for an integration time of 3600.00 s 


Now, assuming you have a pixelized model of your source, calculate the average surface brightness over an area of choice.  

Average surface brightness = $\sum$ (surface brightness within each beam) / $N_{beams}$, so error in average surface brightness will be given by (surface brightness error within a beam) / $\sqrt{N_{beams}}$.  

Then flux density error per beam = (surface brightness error within a beam) $\times \Omega_{beam}$ so error in average surface brightness = (flux density error per beam) / $\Omega_{beam}$ / $\sqrt{N_{beams}}$.

Eg, if I have a cluster model and I'm selecting pixels within a radial bin, first set up a pixel grid and calculate distances from the centre

In [35]:
# load in/create model as an array in variable "model" with resolution "res" in arcmin
#npix=np.shape(model)[0] # assuming it's square
#crpix=npix/2
#x=(np.arange(npix)-crpix)*res
#X, Y=np.meshgrid(x, x)
#R=np.sqrt(X**2+Y**2)

Select pixels within the desired region, in this case an annulus, and average surface brightness (in MJy sr$^{-1}$ here)

In [36]:
#in_region=np.nonzero((R>theta_i) & (R<theta_f))
#mean_SB=np.sum(model[in_region])/float(len(R[in_region])) # MJy sr^-1
#A=len(R[in_region])*(res*60)**2 # arcsec^2

If you have a model, uncomment the above.  For the sake of something simple to demonstrate, just put in some numbers here:

In [37]:
mean_SB=0.1 # MJy sr^-1
A=100. # arcsec^2

Select the band you're interested in (or loop over them), calculate the beam area and hence the error in the mean surface brightness

In [38]:
iband=3
Omega_beam=np.pi*beam_size[iband]**2/4/np.log(2.) # arcsec^2
Nbeams=A/Omega_beam
print('Beam size is %.2f arcsec, area covered by %.2f beams' % (beam_size[iband], Nbeams))
SB_err=sens_per_hour[iband]*1e-9/(Omega_beam*(np.pi/180/3600)**2)/np.sqrt(Nbeams) # convert sensitivity from mJy --> MJy, convert beam solid angle to sr
SNR=mean_SB/SB_err
print('Surface brightness error = %.3g MJy/sr; SNR = %.1f' % (SB_err, SNR))

Beam size is 6.11 arcsec, area covered by 2.37 beams
Surface brightness error = 0.00536 MJy/sr; SNR = 18.6


Alternatively, require an SNR and work out the observing time you need to get to it.  Ie SNR = mean SB / (flux density error per beam) $ \times \Omega_{beam} \times \sqrt{N_{beams}} \Rightarrow$ flux density error per beam = mean SB / SNR $ \times \Omega_{beam} \times \sqrt{N_{beams}}$

In [39]:
des_SNR=50.
flux_err=mean_SB*1e9/des_SNR*(Omega_beam*(np.pi/180/3600)**2)*np.sqrt(Nbeams) # MJy --> mJy and Omega_beam to sr
print('Required flux density error = %.3g mJy/beam for SNR=%.2f' % (flux_err, des_SNR))

Required flux density error = 0.00306 mJy/beam for SNR=50.00


Convert this to an observing time estimate.  Could also use the sensitivity calculator for this.

In [40]:
req_time = (sens_per_hour[iband]/flux_err)**2
print('Sensitivity after 1 hr observing is %.3g mJy/beam; required time to reach %.3g mJy/beam is %.2f hours' % (sens_per_hour[iband], flux_err, req_time))

Sensitivity after 1 hr observing is 0.0082 mJy/beam; required time to reach 0.00306 mJy/beam is 7.19 hours
